In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import sklearn
import os
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import tqdm

from sklearn.preprocessing import *
from sklearn.impute import SimpleImputer

import fuzzywuzzy
from fuzzywuzzy import process

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from header import *

In [ ]:
data_root = initialize_data_root("AY")

In [ ]:
raw_content = pd.read_csv(data_root+'content.csv', dtype = str)

In [ ]:
content = raw_content.fillna('')

In [ ]:
len(content)

2699

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
i=1

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

content['article_content']=content['article_content'].map(lambda s:preprocess(s)) 

In [ ]:
topic_counts = {}

def process_article(article):
    words = set(article.split(' '))
    for word in words:
        if word in topic_counts:
            topic_counts[word] += 1
        else:
            topic_counts[word] = 1 
            
content.apply(lambda x:process_article(x['article_content']), axis = 1)

0       None
1       None
2       None
3       None
4       None
        ... 
2694    None
2695    None
2696    None
2697    None
2698    None
Length: 2699, dtype: object

In [ ]:
len(content)

2699

In [ ]:
filtered_topic_counts = topic_counts.copy()

for i in topic_counts:
    if filtered_topic_counts[i] > 30 and filtered_topic_counts[i] < 2500:
        filtered_topic_counts.pop(i)

In [ ]:
bag = list(filtered_topic_counts.keys())
len(bag)

19656

In [ ]:
content['article_content']

0       raising child dyslexia stir lot emotions may l...
1       raising child dyslexia stir lot emotions may l...
2       comes learning attention issues awareness ever...
3       october learning disabilities adhd dyslexia aw...
4       adhd adhd common condition impacts focus self ...
                              ...                        
2694    good relationship families schools top factor ...
2695    whenever ask veteran educators english languag...
2696    ever asked students read text find understandi...
2697    know importance collaborative partnerships fam...
2698    phonics blending phonics blending way students...
Name: article_content, Length: 2699, dtype: object

In [ ]:
def bag_of_words(data, bag):
    topic_occurence = words_in_texts(bag, data['article_content'])
    
    new_data = pd.DataFrame(data = topic_occurence, columns = bag)
    new_data.insert(0, 'url', data['url'])
        
    return new_data

def words_in_texts(words, texts):
    '''
    Args:
        words (list-like): words to find
        texts (Series): strings to search in
    
    Returns:
        NumPy array of 0s and 1s with shape (n, p) where n is the
        number of texts and p is the number of words.
    '''
    indicator_array = np.array([texts.str.contains(word) for word in words]).T
    return indicator_array.astype('int32')

In [ ]:
content['article_content']

0       raising child dyslexia stir lot emotions may l...
1       raising child dyslexia stir lot emotions may l...
2       comes learning attention issues awareness ever...
3       october learning disabilities adhd dyslexia aw...
4       adhd adhd common condition impacts focus self ...
                              ...                        
2694    good relationship families schools top factor ...
2695    whenever ask veteran educators english languag...
2696    ever asked students read text find understandi...
2697    know importance collaborative partnerships fam...
2698    phonics blending phonics blending way students...
Name: article_content, Length: 2699, dtype: object

In [ ]:
bow_content = words_in_texts(bag, content['article_content'])

In [ ]:
content.shape

(2699, 7)

In [ ]:
bow_content_df = pd.DataFrame(data = bow_content, columns = bag)
bow_content_df.insert(0, 'url', content['url'])

In [ ]:
bow_content_df

,url,numbershas,occasion,reaches,infants,idiomshas,sell,levelhas,fives,blurry,...,layla,morgan,sssiiittt,decodeable,osborn,jjjaamm,armbruster,lehr,phinder,adler
0,/g26ftzdilh/bfunderstanding-dyslexia,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,/learning-attention-issues/getting-started/wha...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/learning-attention-issues/getting-started/wha...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,/learning-attention-issues/child-learning-disa...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,/school-learning/for-educators/partnering-with...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2695,/school-learning/for-educators/partnering-with...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2696,/school-learning/for-educators/teaching-strate...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2697,/school-learning/for-educators/partnering-with...,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [ ]:
def cosine_similar(x,y):
    x_meta = x.drop(['url'])
    y_meta = y.drop(['url'])
    
    return(x_meta@y_meta/((x_meta@x_meta)**0.5*(y_meta@y_meta)**0.5))
    
cosine_similar(bow_content_df.iloc[1,:], bow_content_df.iloc[10,:])

0.49780809060272635

In [ ]:
def find_most_similar_article(url, n):
    article = bow_content_df.loc[bow_content_df['url']==url,:]
    
    others = bow_content_df.loc[bow_content_df['url']!=url,:]
        
    similarities = others.apply(lambda x:cosine_similar(x,article.iloc[0,:]), axis = 1)
    
    top = similarities.nlargest(n).index
    return others.loc[top, "url"]
    
    
    # return(similarities)

In [ ]:
cosine_similar(others.iloc[2,:], article.iloc[0,:])

0.3512649950880297

In [ ]:
find_most_similar_article('/g26ftzdilh/bfunderstanding-dyslexia',3)

1    /g26ftzdilh/blazingly-fast-understanding-dyslexia
5    /learning-attention-issues/child-learning-disa...
6    /learning-attention-issues/child-learning-disa...
Name: url, dtype: object

In [ ]:
bow_content_df.to_csv(data_root+'article_bag_of_words.csv', index = False)